# Universo Fuzzy: Análise e Fuzzificação
Um notebook para definir o universo de discurso, particionar o domínio em funções de pertinência, realizar fuzzificação de amostras e analisar os resultados.

## 1. Definir Universo de Discurso
Defina o intervalo do universo de discurso e as amostras para fuzzificação.

In [ ]:
# Definição do universo de discurso
dominio = (0, 100)  # Intervalo do universo de discurso
samples = [25, 75]  # Amostras para fuzzificação

## 2. Particionar o Domínio em Funções de Pertinência
Implemente funções para gerar parâmetros e calcular graus de pertinência para diferentes tipos de funções fuzzy.

In [ ]:
import numpy as np

# Função para gerar parâmetros para diferentes tipos de funções fuzzy
def generate_params(X, types, n):
    centers = np.linspace(X[0], X[1], n)
    step = (X[1] - X[0]) / (n - 1) if n > 1 else (X[1] - X[0])
    params = []
    for i, t in enumerate(types):
        if t == 'gaussian':
            sigma = step / 2
            params.append([centers[i], sigma])
        elif t == 'triangular':
            a = max(X[0], centers[i] - step)
            b = centers[i]
            c = min(X[1], centers[i] + step)
            params.append([a, b, c])
        elif t == 'trapezoidal':
            a = max(X[0], centers[i] - step)
            b = max(X[0], centers[i] - step / 2)
            c = min(X[1], centers[i] + step / 2)
            d = min(X[1], centers[i] + step)
            params.append([a, b, c, d])
        else:
            raise ValueError(f"Tipo de função '{t}' não suportado!")
    return params

# Função para calcular os graus de pertinência
def calculate_membership(dominio, types, params):
    x = np.linspace(dominio[0], dominio[1], 100)
    results = []
    for func_type, func_params in zip(types, params):
        if func_type == 'gaussian':
            results.append([np.exp(-0.5 * ((val - func_params[0]) / func_params[1]) ** 2) for val in x])
        elif func_type == 'triangular':
            results.append([max(0, min((val - func_params[0]) / (func_params[1] - func_params[0]), 
                                       (func_params[2] - val) / (func_params[2] - func_params[1]))) for val in x])
        elif func_type == 'trapezoidal':
            results.append([max(0, min((val - func_params[0]) / (func_params[1] - func_params[0]), 
                                       1, 
                                       (func_params[3] - val) / (func_params[3] - func_params[2]))) for val in x])
        else:
            raise ValueError(f"Tipo de função desconhecido: {func_type}")
    return x, results

## 3. Fuzzificação de Duas Amostras
Calcule os graus de pertinência para duas amostras específicas usando as funções fuzzy definidas.